## Importing relevant libraries

In [ ]:
import numpy as np
import scipy.linalg as la
import scipy.spatial as spat
from scipy.stats import unitary_group
from scipy.stats import moment
from scipy.stats import skew, kurtosis
from scipy.optimize import curve_fit
from scipy.linalg import norm
import matplotlib.pyplot as plt
import math
from dataclasses import dataclass

# Libraries for implementing the VQD algorithm
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.primitives import Sampler, Estimator
from qiskit_aer import AerSimulator
# from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Sampler, Session, Options
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import RealAmplitudes, TwoLocal, EfficientSU2
from qiskit_algorithms.optimizers import *
from qiskit_algorithms.state_fidelities import ComputeUncompute

from qiskit_algorithms.eigensolvers import EigensolverResult, VQD
from qiskit_algorithms import NumPyMinimumEigensolver, VQE

# Import classical optimizers
from qiskit_algorithms.optimizers import SPSA, P_BFGS, COBYLA, IMFIL, SNOBFIT, NELDER_MEAD, SLSQP, NFT, ADAM, POWELL, GradientDescent, BOBYQA

# Import Statevector and SparsePauliOp
from qiskit.quantum_info import SparsePauliOp, Statevector

# Import noise models
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)

# Import a fake backend and Qiskit simulators and/or noise libraries
from qiskit_aer import AerSimulator
# from qiskit_aer.primitives import Estimator as AerEstimator 
# from qiskit_aer.noise import NoiseModel

## Helper functions

In [ ]:
def get_unitary(matrix, add_half = False):
    """
    Purpose: given a matrix, returns the unitary, hermitian matrix to be diagonalized
    Input: matrix -> the matrix to be diagonalized
    Output: U -> the unitary matrix
            nqubits -> the number of qubis needed to represent the basis of U
            dimension -> the dimension of the original matrix
    """
    assert matrix.ndim == 2, "Error: Only a matrix maybe processed"
    assert matrix.shape[0] == matrix.shape[1], "Error: Only a square matrix maybe processed"

    if np.any(np.transpose(matrix) != matrix):
        matrix_T = np.transpose(matrix)
        matrix = np.dot(matrix_T, matrix)

    ## Finding the dimension of the matrix
    dimension_hermitian = matrix.shape[0]

    ## Finding the number of qubits required to represent the matrix
    nqubits = int(np.ceil(np.log2(dimension_hermitian)))

    ## Construct the relevant matrix
    op_dim = 2 ** nqubits
    op = np.eye(op_dim)
    op[0:dimension_hermitian, 0:dimension_hermitian] = np.copy(matrix)

    if add_half:
        op = op + np.pi * np.eye(op.shape[0])

    U = la.expm(1j*op)
    
    # Get the dimensions of the unitary matrix
    dimension = U.shape[0]

    return U, nqubits, dimension

def find_probability(eigenvector_raw):
    """
    Purpose: Find the probability associated with each basis of an eigenvector
    Input: eigenvector_raw -> Numpy array documenting the number of times each basis is detected within the eigenvector
    Output: eigenvector_prob -> Numpy array documenting the probability of detecting each basis
    """
    count_total = np.sum(eigenvector_raw)
    eigenvector_prob = eigenvector_raw / count_total
    
    return eigenvector_prob

def find_amplitude(eigenvector_prob):
    """
    Purpose: Finding the probability amplitude of each basis using quantum mechanics
    Input: eigenvector_prob -> Numpy array documenting the probability that each basis is measured
    Output: eigenvector -> Numpy array representing the eigenvector
    """
    eigenvector = np.sqrt(eigenvector_prob)
    return eigenvector

def normalize_eigenvector(vector):
    """
    Purpose: Normalizes a vector such that its norm is 1
    Input: vector -> The vector to be normalized
    Output: vector -> The normalized vector
    """
    L2 = np.sum(np.square(vector))
    vector = vector / np.sqrt(L2)

    return vector

def get_similarity(a, b):
    # Function to compute the similarity between 2 zeromodes

    # Get absolute values
    b = np.abs(b)
    
    numerator = np.abs(np.dot(a.conj().T, b))**2
    denominator = np.linalg.norm(a)**2 * np.linalg.norm(b)**2
    
    return numerator / denominator

def get_expectation(matrix, zeromode):
    # Compute the expectation value of the Schlogl operator in the steady-state
    
    # Convert the zeromode into a matrix (column vector)    
    zeromode = np.array(zeromode).reshape(len(zeromode), 1)
    zeromode = np.abs(zeromode) # get rid of all (extraneous) negative values (since this is a PDF)
    
    # Compute the steady-state expectation value
    value = np.dot(matrix, zeromode)
    expectation_value = np.dot(zeromode.T, value)
    
    return expectation_value

def compute_rmsd(list1, list2):
    # Ensure the lists have the same length
    
    if len(list1) != len(list2):
        raise ValueError("Lists must have the same length.")
    
    # Compute the RMSD
    rmsd = np.sqrt(np.mean((np.array(list1) - np.array(list2)) ** 2))
    return rmsd

def get_expectation(matrix, zeromode):
    # Compute the expectation value of the Schlogl operator in the steady-state
    
    # Convert the zeromode into a matrix (column vector)    
    zeromode = np.array(zeromode).reshape(len(zeromode), 1)
    zeromode = np.abs(zeromode) # get rid of all (extraneous) negative values (since this is a PDF)
    
    # Compute the steady-state expectation value
    value = np.dot(matrix, zeromode)
    expectation_value = np.dot(zeromode.T, value)
    
    return expectation_value

def compute_eigenvalues(matrix):
    """
    Computes the two eigenvalues with the highest real parts of a general matrix.

    Parameters:
        matrix (np.ndarray): The matrix for which to compute the eigenvalues.

    Returns:
        list: A list containing the two eigenvalues with the highest real parts [eigenvalue1, eigenvalue2].
    """
    # Compute all eigenvalues
    eigenvalues = np.linalg.eigvals(matrix)
    
    # Sort by real part in descending order and select the two with the lowest real parts
    lowest_two_eigenvalues = sorted(eigenvalues, key=lambda x: x.real, reverse=False)[:2]
    
    return lowest_two_eigenvalues

def compute_relative_errors(exact_eigenvalues, vqd_eigenvalues):
    """
    Computes the relative errors for the first and second eigenvalues for each optimizer.

    Parameters:
        exact_eigenvalues (list): List of the two lowest exact eigenvalues [lambda_1, lambda_2].
        vqd_eigenvalues (dict): Dictionary where keys are optimizer names and values are lists of the two
                                lowest VQD eigenvalues computed by each optimizer.

    Returns:
        dict: Dictionary where each key is an optimizer name and the value is a tuple of the relative errors
              (error_first, error_second).
    """
    # Extract the exact eigenvalues for easy reference
    exact_first, exact_second = exact_eigenvalues
    
    # Initialize a dictionary to store relative errors for each optimizer
    relative_errors = {}
    
    for optimizer, eigenvalues in vqd_eigenvalues.items():
        vqd_first, vqd_second = eigenvalues  # VQD-computed first and second eigenvalues for this optimizer
        
        # Calculate relative errors for the first and second eigenvalues
        error_first = abs((vqd_first - exact_first) / exact_first) if exact_first != 0 else np.nan
        error_second = abs((vqd_second - exact_second) / exact_second) if exact_second != 0 else np.nan
        
        # Store the errors in the dictionary
        relative_errors[optimizer] = (error_first, error_second)
    
    return relative_errors

## VQD implementation

In [ ]:
def run_vqd_eigenvalue_analysis(matrix, optimizer, ansatz_depth, seed, use_seed=False, compute_fidelity=False, \
                                zeromode_exact=None, capture_metrics=False):
    """
    Runs the VQD algorithm to compute the two lowest eigenvalues of the Hermitian form H†H and, if specified,
    calculates fidelity with an exact zeromode. Also captures function calls, gate count, and circuit depth.

    Parameters:
        matrix (np.ndarray): The original matrix H (used to create H†H).
        optimizer (Optimizer): The optimizer to use in VQD.
        ansatz_depth (int): Depth of the ansatz circuit.
        seed (int): Seed for initial point if use_seed is True.
        use_seed (bool): Whether to use a seeded initial point.
        compute_fidelity (bool): Whether to compute fidelity with the exact zeromode.
        zeromode_exact (np.ndarray): The exact zeromode for fidelity calculation if compute_fidelity=True.
        capture_metrics (bool): If True, returns function calls, gate count, and circuit depth.

    Returns:
        tuple: (fidelity, eigenvalues, func_calls, gate_count, circuit_depth), where:
            - eigenvalues: list of the two lowest eigenvalues of H†H,
            - fidelity: computed fidelity if compute_fidelity=True, otherwise None,
            - func_calls: number of function calls (if capture_metrics=True),
            - gate_count: gate count of the ansatz (if capture_metrics=True),
            - circuit_depth: circuit depth (if capture_metrics=True).
    """
    # Determine the number of qubits based on matrix dimension
    dimension = matrix.shape[0]
    num_qubits = int(np.log2(dimension))
    
    # Construct the Hamiltonian
    qub_hamiltonian = SparsePauliOp.from_operator(matrix)

    # Define the ansatz with the specified depth
    ansatz = RealAmplitudes(num_qubits=num_qubits, reps=ansatz_depth)
    
    # Initialize the optimizer and the initial point
    if use_seed:
        np.random.seed(seed)
    initial_point = np.random.uniform(-np.pi, np.pi, ansatz.num_parameters)
    
    # Initialize the Estimator, Sampler, and Fidelity functions
    estimator = Estimator()
    sampler = Sampler()
    fidelity_obj = ComputeUncompute(sampler)
    
    # Run VQD to find the two lowest eigenvalues of the Hermitian form H†H
    vqd = VQD(estimator, fidelity_obj, ansatz, optimizer, k=2, initial_point=initial_point)
    result = vqd.compute_eigenvalues(qub_hamiltonian)
    
    # Extract the two lowest eigenvalues
    eigenvalues = result.eigenvalues
    
    # Compute fidelity if requested
    fidelity = None
    if compute_fidelity and zeromode_exact is not None:
        # Get the zeromode from the VQD result
        optimal_params = result.optimal_points[0]
        final_circuit = ansatz.assign_parameters(optimal_params)
        zeromode_vqd = Statevector.from_instruction(final_circuit).data
        fidelity = get_similarity(zeromode_exact, zeromode_vqd)
    
    # Capture additional metrics if required
    func_calls, gate_count, circuit_depth = None, None, None
    if capture_metrics:
        func_calls = result.cost_function_evals  # Number of function evaluations
        # Total number of gates in the ansatz
        decomposed_ansatz = ansatz.decompose()  
        gate_count_dict = decomposed_ansatz.count_ops()
        gate_count = sum(gate_count_dict.values())
        circuit_depth = decomposed_ansatz.depth()  # Depth of the ansatz circuit

    return fidelity, eigenvalues, func_calls, gate_count, circuit_depth

## Accuracy of eigenvalues and quantum resource requirements

In [ ]:
def vqd_eigenvalue_resource_estimates(matrix, zeromode_classic, optimizers, num_qubits, target_relative_error_threshold, exact_second_eigenvalue):
    """
    Computes the minimum ansatz depth required to achieve a target relative error threshold for the second eigenvalue 
    for a set of optimizers. For each optimizer, at the minimum depth that achieves the relative error threshold, 
    the quantum-computed lowest two eigenvalues of the original matrix H are returned.

    Parameters:
        matrix (np.ndarray): The original matrix H.
        zeromode_classic (np.ndarray): The classical zeromode for fidelity comparison.
        optimizers (list): A list of optimizers to use.
        num_qubits (int): The number of qubits.
        target_relative_error_threshold (float): The relative error threshold for the second eigenvalue.
        exact_second_eigenvalue (float): The exact second eigenvalue of the original matrix H.

    Returns:
        min_depths (dict): Minimum depths for each optimizer to achieve the relative error threshold.
        eigenvalues (dict): Quantum-computed first and second eigenvalues for each optimizer at the minimum depth.
        metrics (dict): Number of function calls, gate count, and circuit depth at the threshold meeting depth.
    """
    # Hermitian form of the matrix for zeromode computation
    hermitian_matrix = matrix.conj().T @ matrix

    # Initialize dictionaries to store results
    min_depths = {}
    eigenvalues = {}
    metrics = {}

    # Loop through each optimizer
    for optimizer in optimizers:
        optimizer_name = optimizer.__class__.__name__
        print(f"\nRunning VQD for optimizer: {optimizer_name}")
        
        current_depth = 1
        max_depth = 10
        converged = False  # Flag to check if convergence occurs
        
        while current_depth <= max_depth:  # Loop up to max_depth
            best_relative_error = np.inf  # Initialize to a high value
            best_run_result = None  # Track best eigenvalues for the run with the best relative error
            best_run_metrics = {}  # Track metrics for the run that meets the threshold
            
            for run in range(10):  # 10 independent VQD runs
                seed = run + 1

                # Run VQD and capture additional metrics
                _, eigenvalues_quantum, func_calls, gate_count, circuit_depth = run_vqd_eigenvalue_analyis(
                    matrix=hermitian_matrix,
                    ansatz_depth=current_depth,
                    optimizer=optimizer,
                    seed=seed,
                    use_seed=True,
                    compute_fidelity=False,
                    capture_metrics=True  # Custom flag to retrieve extra metrics
                )
                
                # Calculate the relative error in the second eigenvalue
                second_eigenvalue = eigenvalues_quantum[1]
                relative_error_in_second_eigenvalue = abs(second_eigenvalue - exact_second_eigenvalue) / abs(exact_second_eigenvalue)

                # Update if this run provides a lower relative error
                if relative_error_in_second_eigenvalue < best_relative_error:
                    best_relative_error = relative_error_in_second_eigenvalue
                    best_run_result = eigenvalues_quantum[:2]  # Store the two eigenvalues
                    best_run_metrics = {
                        "run_number": run + 1,
                        "function_calls": func_calls,
                        "gate_count": gate_count,
                        "circuit_depth": circuit_depth
                    }
                
            # Check if the best relative error in the second eigenvalue meets the threshold
            if best_relative_error <= target_relative_error_threshold:
                min_depths[optimizer_name] = current_depth
                eigenvalues[optimizer_name] = best_run_result
                metrics[optimizer_name] = best_run_metrics
                print(f"Minimum ansatz depth for {optimizer_name} to achieve target relative error threshold: {current_depth}")
                print(f"Best estimate for lowest two eigenvalues at this depth: {best_run_result}")
                print(f"Metrics at threshold: Function calls = {best_run_metrics['function_calls']}, "
                      f"Gate count = {best_run_metrics['gate_count']}, Circuit depth = {best_run_metrics['circuit_depth']}")
                print(f"Relative error threshold met at run number: {best_run_metrics['run_number']}")
                
                converged = True
                break  # Exit if threshold is met

            current_depth += 1  # Increase depth if threshold not met
        
        # If no convergence occurs within max_depth, mark as "did not converge"
        if not converged:
            min_depths[optimizer_name] = "Did not converge"
            eigenvalues[optimizer_name] = "Did not converge"
            metrics[optimizer_name] = "Did not converge"
            print(f"{optimizer_name} did not converge within {max_depth} depths.")
    
    return min_depths, eigenvalues, metrics